# Notebook 2 - SQL Analysis
**Objectif :** Effectuer des agrégations exploratoires sur les données originales (non transformées) via Spark SQL.

In [1]:
# Étape 1 : Initialisation Spark
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, stddev

spark = SparkSession.builder \
    .appName("Fraud-SQL-Analysis") \
    .getOrCreate()

In [2]:
# Étape 2 : Charger le dataset CSV original et créer la vue temporaire (CORRECTION)
# Chemin de base HDFS (hôte:port)
hdfs_base_path = "hdfs://namenode:8020/fraud_data"

# 1. Charger les données d'entraînement (Train)
# Correction de l'URI HDFS : hdfs://namenode:8020/fraud_data/train
train_df = spark.read.parquet(f"{hdfs_base_path}/train")

# 2. Charger les données de test (Test)
test_df = spark.read.parquet(f"{hdfs_base_path}/test")

# 3. Créer la vue temporaire pour l'analyse SQL (en utilisant les données d'entraînement)
train_df.createOrReplaceTempView("train_transactions")
test_df.createOrReplaceTempView("test_transactions")

# Vérification
print("Schéma du DataFrame d'entraînement Parquet :")
train_df.printSchema()
train_df.show(5)

Schéma du DataFrame d'entraînement Parquet :
root
 |-- features: vector (nullable = true)
 |-- label: integer (nullable = true)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[33261.0,-1.46172...|    0|
|[33261.0,-1.19090...|    0|
|[33261.0,1.136010...|    0|
|[33261.0,1.316646...|    0|
|[33262.0,-0.68928...|    0|
+--------------------+-----+
only showing top 5 rows



In [3]:
# Étape 3 : Nombre de transactions par classe 
print("--- Nombre de transactions par classe ---")
df_class_count = spark.sql("SELECT label, COUNT(*) AS count FROM train_transactions GROUP BY label")
df_class_count.show()

--- Nombre de transactions par classe ---
+-----+------+
|label| count|
+-----+------+
|    1|   389|
|    0|227836|
+-----+------+



In [4]:
# 🔹 Étape 4 : Montant moyen et Écart-type par classe 
from pyspark.ml.functions import vector_to_array # Importation de la fonction vector_to_array
from pyspark.sql.functions import col, avg, stddev

print("\n--- Montant moyen et Écart-type par classe ---")

# 1. Conversion du vecteur 'features' en tableau (Array)
df_array = train_df.withColumn(
    "features_array",
    vector_to_array(col("features"))
)

# 2. Extraction de la colonne 'Amount' du tableau
df_with_amount = df_array.withColumn(
    "Amount",
    col("features_array").getItem(29)
)

# 3. Calcul des statistiques
df_avg_amount = df_with_amount.groupBy("label").agg(
    avg(col("Amount")).alias("avg_amount"),
    stddev(col("Amount")).alias("stddev_amount")
)

# 4. Affichage des résultats
df_avg_amount.show()


--- Montant moyen et Écart-type par classe ---
+-----+------------------+------------------+
|label|        avg_amount|     stddev_amount|
+-----+------------------+------------------+
|    1|117.65683804627248| 235.4446450166963|
|    0|  88.2698110921867|247.64371496582802|
+-----+------------------+------------------+

